In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# get the data in zipped format

import requests
from bs4 import BeautifulSoup
import os

url = 'https://data.caida.org/datasets/passive-2019/equinix-nyc/20190117-130000.UTC/'
username = 'pes1202201377@pesu.pes.edu'
password = 'rasgullabros'

response = requests.get(url, auth=(username, password))
soup = BeautifulSoup(response.content, 'html.parser')

links = soup.find_all('a')

os.makedirs('equinix_nyc_data', exist_ok=True)

download_count = 0

for link in links:
    file_name = link.get('href')
    if file_name.endswith('.pcap.gz'):
        file_url = url + file_name
        print(f'Downloading {file_name}')
        file_response = requests.get(file_url, auth=(username, password))
        
        with open(os.path.join('equinix_nyc_data', file_name), 'wb') as f:
            f.write(file_response.content)
        
        download_count += 1
        if download_count == 6:
            break

print("Download complete")


In [2]:
pip install pyshark nest_asyncio scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install gzip

ERROR: Could not find a version that satisfies the requirement gzip (from versions: none)
ERROR: No matching distribution found for gzip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# only unzip

import os
import gzip

input_dir = './equinix_nyc_data'
output_dir = './unzipped_data'

os.makedirs(output_dir, exist_ok=True)

def unzip_gz_files(input_dir, output_dir):
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.pcap.gz'):
            gz_file_path = os.path.join(input_dir, file_name)
            output_file_path = os.path.join(output_dir, file_name[:-3])
            try:
                with gzip.open(gz_file_path, 'rb') as gz_file:
                    with open(output_file_path, 'wb') as output_file:
                        output_file.write(gz_file.read())
                print(f"Unzipped: {gz_file_path} -> {output_file_path}")
            except Exception as e:
                print(f"Error unzipping {gz_file_path}: {e}")

unzip_gz_files(input_dir, output_dir)


In [2]:
pip install scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import scapy.all as scapy
import csv

input_dir = '/kaggle/input/unzipped-pcap-2019/unzipped_data'
chunk_dir = './chunked_data'
csv_dir = './csv_data'

os.makedirs(chunk_dir, exist_ok=True)
os.makedirs(csv_dir, exist_ok=True)

def chunk_pcap(input_pcap, chunk_size_mb, output_dir):
    packets = scapy.rdpcap(input_pcap)
    chunk_data = []
    chunk_size = 0
    chunk_count = 1
    output_file = os.path.join(output_dir, f"chunk_{chunk_count}.pcap")

    for packet in packets:
        packet_size = len(bytes(packet))
        chunk_data.append(packet)
        chunk_size += packet_size / (1024 * 1024)
        
        if chunk_size >= chunk_size_mb:
            scapy.wrpcap(output_file, chunk_data)
            print(f"Saved chunk {chunk_count} as {output_file}")
            chunk_count += 1
            chunk_data = []
            chunk_size = 0
            output_file = os.path.join(output_dir, f"chunk_{chunk_count}.pcap")
    
    if chunk_data:
        scapy.wrpcap(output_file, chunk_data)
        print(f"Saved final chunk {chunk_count} as {output_file}")

def process_pcap_to_csv(chunk_path, csv_path):
    try:
        packets = scapy.rdpcap(chunk_path)
        with open(csv_path, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["Timestamp", "Source", "Destination", "Protocol", "Length"])
            for packet in packets:
                try:
                    if packet.haslayer(scapy.IP):
                        writer.writerow([
                            packet.time,
                            packet[scapy.IP].src,
                            packet[scapy.IP].dst,
                            packet.proto,
                            len(packet),
                        ])
                except AttributeError:
                    continue
        print(f"Processed {chunk_path} -> {csv_path}")
    except Exception as e:
        print(f"Error processing {chunk_path}: {e}")

pcap_files = [file for file in os.listdir(input_dir) if file.endswith('.pcap')]
if pcap_files:
    first_pcap_file = pcap_files[0]
    input_pcap_path = os.path.join(input_dir, first_pcap_file)

    # Step 1: Chunk the pcap file
    chunk_pcap(input_pcap_path, chunk_size_mb=100, output_dir=chunk_dir)

    # Step 2: Process each chunk into a CSV
    for chunk_file in os.listdir(chunk_dir):
        if chunk_file.startswith(first_pcap_file) and chunk_file.endswith('.pcap'):
            chunk_path = os.path.join(chunk_dir, chunk_file)
            csv_file_path = os.path.join(csv_dir, f"{chunk_file}.csv")
            process_pcap_to_csv(chunk_path, csv_file_path)

print("Processing completed for the first .pcap file.")